# Project : Arrests in Maryland
### Enock Gansou, Subhee Rawal

### Part 1: Preparing and Setting Up Map for the Dataset

In [12]:
!pip install folium
import folium
from folium.map import *
import requests
import pandas

arrest_table = pandas.read_csv("http://www.hcbravo.org/IntroDataSci/misc/BPD_Arrests.csv")

arrest_table["race_new"] = arrest_table["sex"]
arrest_table["sex_new"] = arrest_table["race"]
arrest_table["race"] = arrest_table["race_new"]
arrest_table["sex"] = arrest_table["sex_new"]
arrest_table = arrest_table.drop('race_new', 1)
arrest_table = arrest_table.drop('sex_new', 1)

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

arrest_table.head()

,arrest,age,sex,race,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
1,11127013.0,37,M,B,01/01/2011,00:01,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,1 1425,Reckless Endangerment || Hand Gun Violation,SOUTHERN,934.0,Carrollton Ridge,"(39.2814026274, -76.6483635135)",39.281403,-76.648364
2,11126887.0,46,M,B,01/01/2011,00:01,2800 Mayfield Ave,Unknown Offense,NaN,NaN,Unknown Charge,NORTHEASTERN,415.0,Belair-Edison,"(39.3227699160, -76.5735750473)",39.322770,-76.573575
3,11126873.0,50,M,B,01/01/2011,00:04,2100 Ashburton St,79-Other,2100 Ashburton St,1 1106,Reg Firearm:Illegal Possession || Hgv,WESTERN,735.0,Panway/Braddish Avenue,"(39.3117196723, -76.6623546313)",39.311720,-76.662355
4,11126968.0,33,M,B,01/01/2011,00:05,4000 Wilsby Ave,Unknown Offense,1700 Aliceanna St,NaN,Unknown Charge,NORTHERN,525.0,Pen Lucy,"(39.3382885254, -76.6045667070)",39.338289,-76.604567
5,11127041.0,41,M,B,01/01/2011,00:05,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,1 1425,Reckless Endangerment || Handgun Violation,SOUTHERN,924.0,Cherry Hill,"(39.2449886230, -76.6273582432)",39.244989,-76.627358


### Part 2: Generate the Interactive Map with the given dataset

In [13]:
# Generating sample of 300 indivuals
sample = arrest_table.sample(n=300)

In [14]:
import random
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)


# Adding the layers to the map
map_sex = FeatureGroup(name='Sex')
map_race = FeatureGroup(name='Race')
        
# Using Markers to represent arrest location and sex
# Blue Markers: Male
# Pink Markers : Female

for index, elem in sample.iterrows():
        if elem['sex'] == 'M':
            map_sex.add_child(folium.Marker(
                location=[elem['lat'],elem['long']],
                popup="ID #: " + str(elem['arrest']) + ", Location: " + elem['arrestLocation'] + 
                ", District: " + elem['district'] + ", Offense: " + elem['incidentOffense'] +
                "Age: " + str(elem['age']),
                 icon=folium.Icon(icon = 'mars', prefix = 'fa', color='blue'),
            ))
        else:
            map_sex.add_child(folium.Marker(
                location=[elem['lat'],elem['long']],
                popup="ID #: " + str(elem['arrest']) + ", Location: " + elem['arrestLocation'] +
                " , District: " + elem['district'] + ", Offense: " + elem['incidentOffense'] +
                  ", Age: " + str(elem['age']),
                 icon=folium.Icon(icon = 'venus', prefix = 'fa', color='pink'),
            ))
        

# Using Circles to represnt race 
# B - Black
# W - White
# A - Yellow
# U - Green
# I - Olive
# H - Cyan
# NaN - No Circle  

for index,elem in sample.iterrows():
    if elem['race'] == 'B':
        map_race.add_child(folium.Circle(
            radius=200,
            location=[elem['lat'],elem['long']],
            popup="Race : " + str(elem['race']),
            color='black',
            fill=True,
        ))
        
    elif elem['race'] == 'W':
        map_race.add_child(folium.Circle(
            radius=200,
            location=[elem['lat'],elem['long']],
            popup="Race : " + str(elem['race']),
            color='white',
            fill=True,
        ))
        
    elif elem['race'] == 'A':
        map_race.add_child(folium.Circle(
            radius=200,
            location=[elem['lat'],elem['long']],
            popup="Race : " + str(elem['race']),
            color='yellow',
            fill=True,
        ))
        
    elif elem['race'] == 'U':
        map_race.add_child(folium.Circle(
            radius=200,
            location=[elem['lat'],elem['long']],
            popup="Race : " + str(elem['race']),
            color='green',
            fill=True,
        ))
    
    elif elem['race'] == 'I':
        map_race.add_child(folium.Circle(
            radius=200,
            location=[elem['lat'],elem['long']],
            popup="Race : " + str(elem['race']),
            color='olive',
            fill=True,
        ))
        
    elif elem['race'] == 'H':
        map_race.add_child(folium.Circle(
            radius=200,
            location=[elem['lat'],elem['long']],
            popup="Race : " + str(elem['race']),
            color='cyan',
            fill=True,
        ))
        
    

# Adding the layers: sex, race
map_osm.add_child(map_race)
map_osm.add_child(map_sex)
map_osm.add_child(folium.map.LayerControl())



import os
from IPython.core.display import HTML
map_osm.save(os.path.join('map_osm.html'))
map_osm

### Explanation
By looking at the above result, we can observe that most arrests happen around the main city and 
the epicentre of Baltimore. Furthermore, the markers used to reprensent sex shows that the most arrested people in Baltimore were male as compared to female. Finally, in the above map, using the circles, we find out that the most arrested individuals happen to be black.